# Algoritmos de multiplicação

Aprendemos na escola a multiplicar duas matrizes $A \in \mathbb{R}^{m \times p}$ e $B \in \mathbb{R}^{p \times n}$ usando a fórmula $c_{i, j} = \sum \limits_{k = 1}^p a_{i, k} b_{k, j}$. O que não nos ensinam é que há mais duas formas de interpretar a multiplicação.

Para entendê-las, primeiramente vamos olhar a multiplicação de matriz e um vetor.

  - [Multiplicação entre matriz e vetor](#Matriz-x-vetor)
  - [Multiplicação entre matriz e matriz](#Matriz-x-matriz)
  - [Funções já existentes](#Usando-funções-já-implementadas)

## Matriz x vetor

Sejam $A \in \mathbb{R}^{m \times n}$ e $v \in \mathbb{R}^n$. Existem duas formas de interpretarmos $u = A v$:

  - Produto escalar com as linhas de $A$:
  $$
  u = 
  \begin{bmatrix}
  A_1 \cdot v \\
  \vdots \\
  A_m \cdot v
  \end{bmatrix}
  $$
  - Combinação linear das colunas de $A$:
  $$
  u = A^1 v_1 + \cdots + A^n v_n
  $$

### Produto escalar

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

v = [1, 2, 3, 4];

In [ ]:
m, n = size(A)

# Cria o vetor u
u = Vector{Float64}(undef, m)

for i = 1:m
    
    # Aqui calculamos o produto escalar
    u[i] = 0.0
    for j = 1:n
        u[i] = u[i] + A[i, j] * v[j]
    end
    
end

println(u)

### Combinação linear

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

v = [1, 2, 3, 4];

In [ ]:
m, n = size(A)

# Cria o vetor u de zeros
u = Vector{Float64}(undef, m)

for i = 1:m
    u[i] = 0.0
end

# ou
# u = zeros(m)

# Realiza a multiplicacao
for j = 1:n
    
    for i = 1:m
        u[i] = u[i] + A[i, j] * v[j]
    end
    
end

println(u)

### Observações

Observe que a única diferença entre os dois códigos é a troca do laço em $i$ pelo laço em $j$, e a consequente necessidade de criar um vetor inteiro de zeros no início.

Dizemos que a versão de produto escalar é **orientada a linhas** e a versão de combinação linear é **orientada a colunas**. Vamos comparar os tempos de multiplicação para cada código.

In [ ]:
"""
Multiplicacao de matriz e vetor, versao linha.
"""
function matriz_vetor_escalar(A, v)
   
    m, n = size(A)

    # Cria o vetor u
    u = Vector{Float64}(undef, m)

    for i = 1:m

        # Aqui calculamos o produto escalar
        u[i] = 0.0
        for j = 1:n
            u[i] = u[i] + A[i, j] * v[j]
        end

    end
    
    return u
    
end

"""
Multiplicacao de matriz e vetor, versao coluna.
"""
function matriz_vetor_combinacao(A, v)

    m, n = size(A)

    # Cria o vetor u de zeros
    u = zeros(m)
    
    # Realiza a multiplicacao
    for j = 1:n

        for i = 1:m
            u[i] = u[i] + A[i, j] * v[j]
        end

    end
    
    return u
    
end

In [ ]:
A = rand(10, 10)
v = rand(10)

println(@elapsed(matriz_vetor_escalar(A, v)))

println(@elapsed(matriz_vetor_combinacao(A, v)))

In [ ]:
n = 1000

A = rand(n, n)
v = rand(n)

println("Tempo orientacao a linhas (n = $(n)): ", @elapsed(matriz_vetor_escalar(A, v)))
println("Tempo orientacao a colunas (n = $(n)): ", @elapsed(matriz_vetor_combinacao(A, v)))

## Matrix x matriz

  - [Produto escalar](#Interpretação-1:-produto-escalar)
  - [Combinação linear](#Interpretação-2:-por-linha-ou-colunas)
  - Soma de matrizes

### Interpretação 1: produto escalar

Forma mais tradicional. Basta observarmos que se $C = A \cdot B$, então
$$
c_{i, j} = \sum \limits_{k = 1}^p a_{i, k} b_{k, j} = A_i \odot B^j
$$

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [ ]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Escolhe i e j
i = 2
j = 3

# Calcula o somatorio
cij = 0

for k = 1:p
    cij = cij + A[i, k] * B[k, j]
end

println("O elemento C_$(i),$(j) = $(cij).")

Experimente:

  - Modificar os valores das variáveis `i` e `j`
  - Colocar valores de `i` e `j` maiores que a dimensão das matrizes
  - Definir `p` como `mB` ao invés de `nA`

Os elementos $c_{i, j}$ têm que ser calculados para todo $i$ e $j$, e para isso usamos 2 laços do tipo `for`

In [ ]:
C = Array(Float64, m, n)

for i = 1:m
    for j = 1:n
        
        # Aqui copiamos o codigo de cij
        # Calcula o somatorio
        C[i, j] = 0

        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        
    end
end

display(C)

Experimente trocar as ordem dos laços `for` nas linhas 3 e 4. A única diferença é na forma como calculamos os elementos de $C$.

A função abaixo implementa a multiplicação usando essa ideia.

In [ ]:
function multiplica_mat_1(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = Array(Float64, m, n)

    for i = 1:m
        for j = 1:n

            # Aqui copiamos o codigo de cij
            # Calcula o somatorio
            C[i, j] = 0

            for k = 1:p
                C[i, j] = C[i, j] + A[i, k] * B[k, j]
            end

        end
    end
    
    return C
    
end

In [ ]:
multiplica_mat_1(A, B)

Experimentos com matrizes aleatórias grandes.

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_1(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_1(A1, B1)))

Observe que quando **dobramos** a dimensão das matrizes, o tempo é **multiplicado por 8**. Dizemos que o custo computacional da multiplicação tradicional é $O(n^3)$, onde $n$ é a dimensão da matriz.

## Interpretação 2: por linha ou colunas

A segunda interpretação ocorre quando olhamos a forma de uma coluna ou linha de $C$: $C_i$ ou $C^j$.
$$
C = A \cdot B = 
\begin{bmatrix}
A \cdot B^1 & \cdots & A \cdot B^n
\end{bmatrix}
=
\begin{bmatrix}
A_1 \cdot B\\ \vdots\\ A_m \cdot B
\end{bmatrix}
$$
onde $A \cdot B^j$ representa o produto entre a matriz $A$ e a coluna $j$ de $B$, e $A_i \cdot B$ é o produto entre a linha $i$ de $A$ e a matriz $B$ inteira.

### Calculando $C^j$

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [ ]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Construindo a coluna j.
# Precisa colocar zeros para somar.
cj = zeros(Float64, m)

j = 3

for i = 1:m
    
    for k = 1:p
    
        cj[i] = cj[i] + A[i, k] * B[k, j]
        
    end
    
end

println("C^$(j) = ", cj)

As duas versões desse código são apresentadas abaixo.

In [ ]:
"""
Esta funcao multiplica duas matrizes, construindo a matriz produto por **colunas**.
"""
function multiplica_mat_2_col_ik(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = zeros(Float64, m, n)
    
    for j = 1:n
        
        for i = 1:m

            for k = 1:p
                
                # Atencao aqui! Temos que trocar cj por C[:, j]!
                # Ou seja, a j-esima coluna de C
                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C
    
end

"""
Esta funcao multiplica duas matrizes, construindo a matriz produto por **colunas**.

Nesta versao, os lacos associados com `i` e `k` foram trocados de ordem: primeiro `k` e
depois `i`.
"""
function multiplica_mat_2_col_ki(A, B)
    
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB
    
    C = zeros(Float64, m, n)
    
    for j = 1:n
        
        for k = 1:p

            for i = 1:m
                
                # Atencao aqui! Temos que trocar cj por C[:, j]!
                # Ou seja, a j-esima coluna de C
                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C
    
end

In [ ]:
multiplica_mat_2_col_ik(A, B)

In [ ]:
multiplica_mat_2_col_ki(A, B)

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_col_ik(A1, B1)))

Experimentos para quando trocamos os laços `for` de ordem

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_col_ki(A1, B1)))

**Por que** a segunda função foi duas vezes mais rápida que a primeira?

### Calculando $C_i$

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [ ]:
# Comandos para obter as dimensoes das matrizes
mA, nA = size(A)
mB, nB = size(B)

println("Dimensao A: $(mA) x $(nA).")
println("Dimensao B: $(mB) x $(nB).")

m = mA
p = nA # ou mB
n = nB

# Construindo a linha i.
# Precisa colocar zeros para somar.
ci = zeros(Float64, n)

i = 2

for j = 1:n
    
    for k = 1:p
    
        ci[j] = ci[j] + A[i, k] * B[k, j]
        
    end
    
end

println("C_$(i) = ", ci)

As duas versões desse código são apresentadas abaixo.

In [ ]:
function multiplica_mat_2_lin_jk(A, B)
   
    # Comandos para obter as dimensoes das matrizes
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB

    # Ja constroi C com zeros
    C = zeros(Float64, m, n)    
    
    for i = 1:m

        for j = 1:n

            for k = 1:p

                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C    
    
end

function multiplica_mat_2_lin_kj(A, B)
   
    # Comandos para obter as dimensoes das matrizes
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB

    # Ja constroi C com zeros
    C = zeros(Float64, m, n)    
    
    for i = 1:m

        for k = 1:p

            for j = 1:n

                C[i, j] = C[i, j] + A[i, k] * B[k, j]

            end

        end
        
    end
    
    return C    
    
end

In [ ]:
multiplica_mat_2_lin_jk(A, B)

In [ ]:
multiplica_mat_2_lin_kj(A, B)

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_lin_jk(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_lin_jk(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_lin_jk(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_lin_jk(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_lin_jk(A1, B1)))

Experimentos para quando trocamos os laços `for` de ordem

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_2_lin_kj(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_2_lin_kj(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_2_lin_kj(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_2_lin_kj(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_2_lin_kj(A1, B1)))

## Interpretação 3 - soma de matrizes

A terceira interpretação ocorre quando montamos $C$ aos poucos: por somas de $p$ matrizes de posto 1.
$$
C = A \cdot B = A^1 B_1 + A^2 B_2 + \cdots + A^p B_p
$$
onde $A^k B_k$ representa o produto entre um vetor coluna de $A$ e um vetor linha de $B$. O resultado desse produto é uma matriz $m \times n$.

In [ ]:
A = [1 2 3 4;
     5 6 7 8];

B = [1 2 1;
     3 4 1;
     5 6 1;
     7 8 1];

In [ ]:
function multiplica_mat_3(A, B)

    # Comandos para obter as dimensoes das matrizes
    mA, nA = size(A)
    mB, nB = size(B)

    if nA != mB
        
        println("Erro. Dimensoes incompatíveis.")
        
        return nothing
        
    end
    
    m = mA
    p = nA # ou mB
    n = nB

    # Ja constroi C com zeros
    C = zeros(Float64, m, n)        
    
    for k = 1:p
        
        for j = 1:n
            
            for i = 1:m
                
                C[i, j] = C[i, j] + A[i, k] * B[k, j]
                
            end
            
        end
        
    end
    
    return C
    
end

In [ ]:
multiplica_mat_3(A, B)

In [ ]:
A1 = rand(100, 100)
B1 = rand(100, 100)

println("Tempo para  100: ", @elapsed(multiplica_mat_3(A1, B1)))

A1 = rand(200, 200)
B1 = rand(200, 200)

println("Tempo para  200: ", @elapsed(multiplica_mat_3(A1, B1)))

A1 = rand(400, 400)
B1 = rand(400, 400)

println("Tempo para  400: ", @elapsed(multiplica_mat_3(A1, B1)))

A1 = rand(800, 800)
B1 = rand(800, 800)

println("Tempo para  800: ", @elapsed(multiplica_mat_3(A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)

println("Tempo para 1000: ", @elapsed(multiplica_mat_3(A1, B1)))

## Usando funções já implementadas

Apesar de todo nosso esforço, existem bibliotecas já prontas que são muito mais eficientes que as nossas implementações: a função `gemm`da bilbioteca BLAS e a operação `*` de Julia.

In [ ]:
using Base.LinAlg.BLAS

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)
println("Tempo da BLAS para 1000: ", @elapsed(gemm('N', 'N', 1.0, A1, B1)))

A1 = rand(1000, 1000)
B1 = rand(1000, 1000)
println("Tempo de *    para 1000: ", @elapsed(A1 * B1))